### Step 1: Import Libraries

In [ ]:
import pandas as pd
import json
import numpy as np
import csv
from datetime import datetime
import os
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import folium
from folium.plugins import MarkerCluster
from sklearn.cluster import KMeans
import numpy as np
from shapely.geometry import MultiPoint
from shapely.ops import unary_union
import matplotlib.pyplot as plt

In [ ]:
os.mkdir('Trip Splitting')

NameError: name 'os' is not defined

### Step 2: Load Data

In [5]:
# Directory and file name pattern
directory = 'Data/clean_data/
file_base = 'clean_2024-01-'

# Generate the list of file names
files_names = [f'{file_base}{i}.json' for i in range(25, 32)] # last week of January 

# Generate the list of file directories
files = [os.path.join(directory, file_name) for file_name in files_names]

data = []

In [6]:
# Load data from files and drop time_diff column
for file_name in files:
    with open(file_name, 'r') as file:
        for line in file:
            data.append(json.loads(line))

df = pd.DataFrame(data)

if 'time_diff' in df.columns:
    df.drop('time_diff', axis=1, inplace=True)

df = df.rename(columns={'time': 'trip_time'})

df.dropna(inplace=True)

In [8]:
# Load pooling data
pooling_file_path = 'Data/careems_cata/anon_pooling_jan_24_amman.csv' 
df_pooling = pd.read_csv(pooling_file_path)

In [ ]:
# Load merchant data
merchant_file_path = 'Data/careems_cata/order_merchant_id_anon.parquet'
merchants_df = pd.read_parquet(merchant_file_path)

#### Left join merchants on the pings df

In [12]:
# Merge merchants_df with df_pooling on 'order_id'
merged_pooling = df_pooling.merge(merchants_df[['order_id', 'merchant_id']], on='order_id', how='left')

# Merge with df on 'trip_id' 
df = df.merge(merged_pooling[['booking_id', 'merchant_id']], left_on='trip_id', right_on='booking_id', how='left')

df = df.drop(columns=['booking_id'])

# Check how many 'merchant_id' values are null 
print(df['merchant_id'].isnull().sum())

0


### Step 3: Stationarity Detection

#### Find all stationary intervals over each trip

In [18]:
def detect_stationary_intervals(df):
    results = []
    for index, row in df.iterrows():
        trip_id = row['trip_id']
        lats = np.array(row['lats'])
        lngs = np.array(row['lngs'])
        time_gap = np.array(row['time_gap'])
        
        # Compute differences between consecutive points directly from lat/lng arrays
        lat_diff = np.abs(np.diff(lats))
        lng_diff = np.abs(np.diff(lngs))

        # Identify stationary intervals
        stationary_indices = np.where((lat_diff == 0) & (lng_diff == 0))[0]
        
        # Group consecutive stationary indices into intervals
        if len(stationary_indices) > 0:
            start_idx = stationary_indices[0]
            for i in range(1, len(stationary_indices)):
                # If the current index is not consecutive, close the interval
                if stationary_indices[i] != stationary_indices[i - 1] + 1:
                    end_idx = stationary_indices[i - 1]
                    # Save the interval only if it has more than one index
                    if end_idx > start_idx:
                        interval = {
                            "trip_id": trip_id,
                            "start_idx": int(start_idx),
                            "end_idx": int(end_idx)+1,
                            "start_lat": float(lats[start_idx]),
                            "start_lng": float(lngs[start_idx]),
                            "end_lat": float(lats[end_idx+1]),
                            "end_lng": float(lngs[end_idx+1]),
                            "time_elapsed": float(time_gap[end_idx+1] - time_gap[start_idx])
                        }
                        results.append(interval)
                    # Start a new interval
                    start_idx = stationary_indices[i]
            # Add the last interval if it has more than one index
            end_idx = stationary_indices[-1]
            if end_idx > start_idx:
                interval = {
                    "trip_id": trip_id,
                    "start_idx": int(start_idx),
                    "end_idx": int(end_idx)+1,
                    "start_lat": float(lats[start_idx]),
                    "start_lng": float(lngs[start_idx]),
                    "end_lat": float(lats[end_idx+1]),
                    "end_lng": float(lngs[end_idx+1]),
                    "time_elapsed": float(time_gap[end_idx+1] - time_gap[start_idx])
                }
                results.append(interval)

    results_df = pd.DataFrame(results)
    return results_df

stationary_df_pre_truncation = detect_stationary_intervals(df)

Stationary intervals saved to stationary_intervals_all_trips.csv


#### Truncate trips where a stationary interval exists at the end of trip

In [19]:

def truncate_trips_if_stationary(df, stationary_df):
    truncated_trips = []

    # Iterate over each trip in the df
    for index, row in df.iterrows():
        trip_id = row['trip_id']
        lats = row['lats']
        lngs = row['lngs']
        time_gap = row['time_gap']
        dist_gap = row['dist_gap']
        dist = row['dist']
        trip_time = row['trip_time']

        # Get the last coordinates of the trip
        last_lat, last_lng = lats[-1], lngs[-1]

        # Find the corresponding stationary intervals for this trip
        intervals = stationary_df[stationary_df['trip_id'] == trip_id]

        if not intervals.empty:
            # Get the last stationary interval for this trip
            last_interval = intervals.iloc[-1]

            interval_lat, interval_lng = last_interval['end_lat'], last_interval['end_lng']
            time_elapsed = last_interval['time_elapsed']

            # Check if the last stationary interval's coordinates match the last trip coordinates
            if (last_lat == interval_lat and last_lng == interval_lng) and time_elapsed > 10:
                # Keep only the first part of the stationary segment
                truncated_trip = {
                    'trip_id': trip_id,
                    'lats': lats[:last_interval['start_idx'] + 1],
                    'lngs': lngs[:last_interval['start_idx'] + 1],
                    'time_gap': time_gap[:last_interval['start_idx'] + 1],
                    'dist_gap': dist_gap[:last_interval['start_idx'] + 1],
                    'dist': dist,  # Keep the original distance
                    'trip_time': time_gap[last_interval['start_idx']],  # Adjusted trip time
                    'driverID': row['driverID'],
                    'weekID': row['weekID'],
                    'timeID': row['timeID'],
                    'dateID': row['dateID'],
                    'merchant_id': row['merchant_id']
                }
                truncated_trips.append(truncated_trip)
            else:
                # If no truncation is needed, keep the original trip
                truncated_trips.append(row.to_dict())
        else:
            # If no stationary intervals exist, keep the original trip
            truncated_trips.append(row.to_dict())

    # Convert the list of truncated trips back into a df
    truncated_df = pd.DataFrame(truncated_trips)

    # Ensure all columns match the original schema
    for column in df.columns:
        if column not in truncated_df.columns:
            truncated_df[column] = None  # Add missing columns with default None values

    # Reorder columns to match the original DataFrame
    truncated_df = truncated_df[df.columns]

    return truncated_df

truncated_df = truncate_trips_if_stationary(df, stationary_df_pre_truncation)


Truncated trips saved to 'truncated_trips.csv'


#### Find new stationary intervals post truncation and longest interval for each trip

In [20]:
def detect_stationary_intervals(df):
    results = []
    for index, row in df.iterrows():
        trip_id = row['trip_id']
        lats = np.array(row['lats'])
        lngs = np.array(row['lngs'])
        time_gap = np.array(row['time_gap'])
        
        # Compute differences between consecutive points directly from lat/lng arrays
        lat_diff = np.abs(np.diff(lats))
        lng_diff = np.abs(np.diff(lngs))

        # Identify stationary intervals 
        stationary_indices = np.where((lat_diff == 0) & (lng_diff == 0))[0]
        
        # Group consecutive stationary indices into intervals
        if len(stationary_indices) > 0:
            start_idx = stationary_indices[0]
            for i in range(1, len(stationary_indices)):
                # If the current index is not consecutive, close the interval
                if stationary_indices[i] != stationary_indices[i - 1] + 1:
                    end_idx = stationary_indices[i - 1]
                    # Save the interval only if it has more than one index
                    if end_idx > start_idx:
                        interval = {
                            "trip_id": trip_id,
                            "start_idx": int(start_idx),
                            "end_idx": int(end_idx)+1,
                            "start_lat": float(lats[start_idx]),
                            "start_lng": float(lngs[start_idx]),
                            "end_lat": float(lats[end_idx+1]),
                            "end_lng": float(lngs[end_idx+1]),
                            "time_elapsed": float(time_gap[end_idx+1] - time_gap[start_idx])
                        }
                        results.append(interval)
                    # Start a new interval
                    start_idx = stationary_indices[i]
            # Add the last interval if it has more than one index
            end_idx = stationary_indices[-1]
            if end_idx > start_idx:
                interval = {
                    "trip_id": trip_id,
                    "start_idx": int(start_idx),
                    "end_idx": int(end_idx)+1,
                    "start_lat": float(lats[start_idx]),
                    "start_lng": float(lngs[start_idx]),
                    "end_lat": float(lats[end_idx+1]),
                    "end_lng": float(lngs[end_idx+1]),
                    "time_elapsed": float(time_gap[end_idx+1] - time_gap[start_idx])
                }
                results.append(interval)

    results_df = pd.DataFrame(results)
    return results_df

stationary_df = detect_stationary_intervals(truncated_df)

Stationary intervals saved to stationary_intervals_all_trips_truncated.csv


In [21]:
df=truncated_df

In [22]:
def extract_longest_stationary_interval(stationary_df):
    # Find the longest stationary interval for each trip_id based on time_elapsed
    longest_intervals = stationary_df.loc[stationary_df.groupby('trip_id')['time_elapsed'].idxmax()]

    longest_intervals.reset_index(drop=True, inplace=True)
    
    return longest_intervals

longest_intervals_df = extract_longest_stationary_interval(stationary_df)

##### Combined df with pooling coordinates and longest stationary interval coordinates

In [23]:
def create_combined(longest_intervals_df, pooling_df):
    
    # Rename 'booking_id' to 'trip_id' in pooling_df for consistency
    pooling_df = pooling_df.rename(columns={'booking_id': 'trip_id'})
    
    combined_df = (
        longest_intervals_df[['trip_id', 'start_lat', 'start_lng']]
        .merge(pooling_df[['trip_id', 'pickup_latitude', 'pickup_longitude']], on='trip_id', how='inner')
    )

    # Rename columns for clarity in the combined data
    combined_df = combined_df.rename(columns={
        'start_lat': 'stationary_df_lat',
        'start_lng': 'stationary_df_lng',
        'pickup_latitude': 'pooling_df_lat',
        'pickup_longitude': 'pooling_df_lng'
    })   
    return combined_df

combined_df = create_combined(longest_intervals_df, df_pooling)

combined_df = combined_df.merge(df[['trip_id', 'merchant_id']], on='trip_id', how='left')

#### Update incorrect pooling coordinates

In [25]:
def check_inconsistent_groups_with_tolerance_fix(combined_df):
    
    # Lists to store inconsistent groups and detailed non-unique counts
    inconsistent_groups = []
    nonunique_info = []

    for merchant_id, group in combined_df.groupby('merchant_id'):
        # Extract latitude and longitude values for the group
        lat_values = group['pooling_df_lat'].values
        lng_values = group['pooling_df_lng'].values

        # Count occurrences of each latitude and longitude
        lat_counts = pd.Series(lat_values).value_counts()
        lng_counts = pd.Series(lng_values).value_counts()

        # Identify the majority (most common) latitude and longitude
        majority_lat = lat_counts.idxmax()
        majority_lng = lng_counts.idxmax()

        # Filter out the non-majority coordinates
        non_majority_lats = lat_counts[lat_counts.index != majority_lat]
        non_majority_lngs = lng_counts[lng_counts.index != majority_lng]

        # If there are non-majority coordinates, store the information
        if not non_majority_lats.empty or not non_majority_lngs.empty:
            inconsistent_groups.append(group)

            # Store the non-unique information for this merchant_id
            nonunique_info.append({
                'merchant_id': merchant_id,
                'non_majority_lats': non_majority_lats.to_dict(),  # Non-majority latitudes and their counts
                'non_majority_lngs': non_majority_lngs.to_dict(),  # Non-majority longitudes and their counts
                'group_size': len(group)
            })

    # Combine all inconsistent groups into a single DataFrame
    if inconsistent_groups:
        print(f"Number of inconsistent groups: {len(inconsistent_groups)}")
        non_majority_df = pd.DataFrame(nonunique_info)
    else:
        print("No inconsistencies found.")

    return non_majority_df

non_majority_df = check_inconsistent_groups_with_tolerance_fix(combined_df)


Inconsistent groups saved to merchant_lat_lng_mismatch.csv
Number of inconsistent groups: 987


In [26]:
# Change incorrect pooling coordinates in combined_df to the majority coordinates for each merchant
def correct_pooling_coordinates_with_nonmajority(non_majority_df, combined_df):

    # Iterate over each merchant in the non-majority df
    for _, row in non_majority_df.iterrows():
        merchant_id = row['merchant_id']

        # Get the majority latitude and longitude for this merchant from combined_df
        majority_lat = combined_df[combined_df['merchant_id'] == merchant_id]['pooling_df_lat'].mode()[0]
        majority_lng = combined_df[combined_df['merchant_id'] == merchant_id]['pooling_df_lng'].mode()[0]

        # Find the trip_ids with non-majority coordinates
        merchant_group = combined_df[combined_df['merchant_id'] == merchant_id]
        incorrect_trips = merchant_group[
            (~np.isclose(merchant_group['pooling_df_lat'], majority_lat, atol=0.001)) |
            (~np.isclose(merchant_group['pooling_df_lng'], majority_lng, atol=0.001))
        ]

        # Update the coordinates in combined_df for the incorrect trips
        for trip_id in incorrect_trips['trip_id']:
            combined_df.loc[combined_df['booking_id'] == trip_id, ['pooling_df_lat', 'pooling_df_lat']] = [
                majority_lat, majority_lng
            ]

    return combined_df


combined_df = correct_pooling_coordinates_with_nonmajority(non_majority_df, combined_df)


Corrected 531 trip coordinates.


#### Extract non-matching coordinates and drop them from df and longest stationary df

In [27]:
def haversine(lat1, lon1, lat2, lon2):
    R = 6371.0

    # Convert latitude and longitude from degrees to radians
    lat1_rad = np.radians(lat1)
    lon1_rad = np.radians(lon1)
    lat2_rad = np.radians(lat2)
    lon2_rad = np.radians(lon2)

    # Compute differences
    dlat = lat2_rad - lat1_rad
    dlon = lon2_rad - lon1_rad

    # Apply Haversine formula
    a = np.sin(dlat / 2.0)**2 + np.cos(lat1_rad) * np.cos(lat2_rad) * np.sin(dlon / 2.0)**2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
    distance = R * c

    return distance

def extract_non_matching_coordinates(combined_df):
    # Identify rows where the stationary coordinates do not match pooling coordinates
    non_matching_df = combined_df[
        (combined_df['stationary_df_lat'] != combined_df['pooling_df_lat']) |
        (combined_df['stationary_df_lng'] != combined_df['pooling_df_lng'])
    ]

    # calculate the distance between the coordinates
    non_matching_df['distance_km'] = haversine(
        non_matching_df['stationary_df_lat'],
        non_matching_df['stationary_df_lng'],
        non_matching_df['pooling_df_lat'],
        non_matching_df['pooling_df_lng']
    )

    return non_matching_df

non_matching_entries = extract_non_matching_coordinates(combined_df)


C:\Users\nadab\AppData\Local\Temp\ipykernel_26592\226864586.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  non_matching_df['distance_km'] = haversine(


Non-matching coordinate entries saved to non_matching_coordinates.csv
trip_id              10071
stationary_df_lat    10071
stationary_df_lng    10071
pooling_df_lat       10071
pooling_df_lng       10071
merchant_id          10071
distance_km          10071
dtype: int64


In [28]:
# Drop non-matching trips from `df` and `longest_intervals_df`
def drop_non_matching_trips(df, longest_intervals_df, non_matching_entries):
    # Extract the trip IDs from non-matching entries
    non_matching_trip_ids = non_matching_entries['trip_id'].unique()

    # Drop these trips from both dfs
    df_filtered = df[~df['trip_id'].isin(non_matching_trip_ids)]
    longest_intervals_filtered = longest_intervals_df[~longest_intervals_df['trip_id'].isin(non_matching_trip_ids)]

    return df_filtered, longest_intervals_filtered


df, longest_intervals_df = drop_non_matching_trips(df, longest_intervals_df, non_matching_entries)
print("Filtered DataFrames saved.")

Filtered DataFrames saved.


### Step 4: Splitting trips into 3 segments (Driver-to-Merchant, Wait-Time-at-Merchant, Merchant-to-Customer)

#### Split trips based on stationary intervals matching coordinates from pooling

In [30]:

def adjust_gaps_with_original_pattern(original_gaps, new_gaps):
    """
    Adjust new gaps based on the pattern of change in the original gaps.
    The new gaps will start from 0 but follow the same incremental differences.
    """
    if len(original_gaps) <= 1 or len(new_gaps) == 0:
        return new_gaps  # No meaningful adjustment needed

    # Calculate the incremental changes (differences) in the original gaps
    original_diffs = np.diff(original_gaps)

    # Start the new gaps from 0
    adjusted_gaps = [0]

    # Apply the original differences to the new gaps
    for i in range(1, len(new_gaps)):
        diff = original_diffs[(i - 1) % len(original_diffs)]
        adjusted_gaps.append(adjusted_gaps[-1] + diff)

    return adjusted_gaps

def segment_trips(df, longest_intervals_df):
    road_list = []
    second_segment_list = []

    # Iterate over each entry in the df dataframe
    for index, row in df.iterrows():
        trip_id = row['trip_id']
        interval = longest_intervals_df[longest_intervals_df['trip_id'] == trip_id]
        lats = row['lats']
        lngs = row['lngs']
        time_gap = row['time_gap']
        dist_gap = row['dist_gap']

        if not interval.empty:
            start_idx = interval['start_idx'].values[0]
            end_idx = interval['end_idx'].values[0]

            # First segment
            first_segment = {
                'trip_id': trip_id,
                'time_gap': time_gap[:start_idx],
                'dist': dist_gap[start_idx - 1] if start_idx > 0 else 0,
                'trip_time': time_gap[start_idx - 1] if start_idx > 0 else 0,
                'driverID': row['driverID'],
                'weekID': row['weekID'],
                'timeID': row['timeID'],
                'dateID': row['dateID'],
                'dist_gap': dist_gap[:start_idx],
                'lats': lats[:start_idx],
                'lngs': lngs[:start_idx],
                'time_offset': 0,
                'segmentID': 1
            }

            # Second segment (stationary)
            stationary_time_gap = time_gap[start_idx:end_idx + 1]
            stationary_dist_gap = dist_gap[start_idx:end_idx + 1]

            stationary_time_gap = [time - stationary_time_gap[0] for time in stationary_time_gap]
            stationary_dist_gap = [dist - stationary_dist_gap[0] for dist in stationary_dist_gap]

            second_segment = {
                'trip_id': trip_id,
                'time_gap': stationary_time_gap,
                'dist': stationary_dist_gap[-1] if len(stationary_dist_gap) > 0 else 0,
                'trip_time': stationary_time_gap[-1] if len(stationary_time_gap) > 0 else 0,
                'driverID': row['driverID'],
                'weekID': row['weekID'],
                'timeID': row['timeID'],
                'dateID': row['dateID'],
                'dist_gap': stationary_dist_gap,
                'lats': lats[start_idx:end_idx + 1],
                'lngs': lngs[start_idx:end_idx + 1],
                'time_offset': time_gap[start_idx],
                'merchant': row['merchant_id']
            }

            # Third segment (after stationary)
            new_time_gap = time_gap[end_idx + 1:]
            new_dist_gap = dist_gap[end_idx + 1:]

            # Adjust the gaps using the pattern from the original trip
            adjusted_time_gap = adjust_gaps_with_original_pattern(time_gap[end_idx + 1:], new_time_gap)
            adjusted_dist_gap = adjust_gaps_with_original_pattern(dist_gap[end_idx + 1:], new_dist_gap)
            time_offset3 = time_gap[end_idx + 1] if len(time_gap) > end_idx + 1 else 0

            third_segment = {
                'trip_id': trip_id,
                'time_gap': adjusted_time_gap,
                'dist': adjusted_dist_gap[-1] if len(adjusted_dist_gap) > 0 else 0,
                'trip_time': adjusted_time_gap[-1] if len(adjusted_time_gap) > 0 else 0,
                'driverID': row['driverID'],
                'weekID': row['weekID'],
                'timeID': row['timeID']+round((time_offset3/60),1),
                'dateID': row['dateID'],
                'dist_gap': adjusted_dist_gap,
                'lats': lats[end_idx + 1:],
                'lngs': lngs[end_idx + 1:],
                'time_offset': time_offset3,
                'segmentID': 3
            }
        else:
            print(f"No interval found for trip_id: {trip_id}")

        # Add segments to their respective lists
        if len(first_segment['time_gap']) > 1:
            road_list.append(first_segment)

        if len(third_segment['time_gap']) > 1:
            road_list.append(third_segment)

        if len(second_segment['time_gap']) > 1:
            second_segment_list.append(second_segment)

    # Convert the lists to DataFrames
    road_df = pd.DataFrame(road_list)
    second_segment_df = pd.DataFrame(second_segment_list)

    return road_df, second_segment_df

road_df, second_segment_df = segment_trips(df, longest_intervals_df)

No interval found for trip_id: 25049734158c2944703546bdc2cbe5740170230779e858db275cb6512aaa0047
No interval found for trip_id: 4ad4bdf931291e3f2176bef563091893e0bde118f28a2ef5b172ece38949ed38
No interval found for trip_id: 5057cb61e831386cd034d24462a4f6ecebecb3606bc2d39d4c991fd04b56e413
No interval found for trip_id: 620950188eebce63b6bf5be22cb280dbb8d75b5cef8904b6de30bb892d2a1c3a
No interval found for trip_id: 64186edda7362e34c7598fe8af53c01cde6071b3d932380eaa30299419712632
No interval found for trip_id: 07a2f3cdf28096c27c4c2b81bcfafe30a96095097f6e96e79878f714a9ec648c
No interval found for trip_id: 6299a27e7d370d6904a05073fdd554e0efbf09f0d9458d7e0f677e75644c2e34
No interval found for trip_id: 65d99e964c50a5eed233fe9a7864114c1d91cf6b6ba542bbf4906a998d4df27f
No interval found for trip_id: 7d71d55894365ccad46d7e7c533997c58e200cc235919b0c817944af8b5226a1
No interval found for trip_id: 01ff05c850fb030d8174a48d70a7327747dd0177030bc5cad10f5c9d25b95b33
No interval found for trip_id: ec0286dc1

In [33]:
# Remove trips with single 3rd segment occurences
single_occurrence_trips = road_df['trip_id'].value_counts()
single_occurrence_trips = single_occurrence_trips[single_occurrence_trips == 1].index

filtered_single_trips_df = road_df[road_df['trip_id'].isin(single_occurrence_trips)]

filtered_single_trips_segment3_df = filtered_single_trips_df[filtered_single_trips_df['segmentID'] == 1]

# Remove these trips from road_df
road_df = road_df[~road_df['trip_id'].isin(filtered_single_trips_segment3_df['trip_id'])]


for date_id in range(24, 31):
    # Filter road_df and second_segment_df for the current dateID
    road_df_filtered = road_df[road_df['dateID'] == date_id]
    second_segment_df_filtered = second_segment_df[second_segment_df['dateID'] == date_id]
    
    # Convert to JSON format
    road_json = road_df_filtered.to_dict(orient='records')
    second_seg_json = second_segment_df_filtered.to_dict(orient='records')

    # Generate filenames with dateID + 1 
    road_file_name = f'Segmented Trips/Segmented_Trips_01_{date_id + 1}.json'
    segment_file_name = f'Segmented Trips/Merchants_Segments_01_{date_id + 1}.json'

    # Save files
    with open(road_file_name, 'w') as file:
        for json_obj in road_json:
            json.dump(json_obj, file)
            file.write('\n')

    with open(segment_file_name, 'w') as file:
        for json_obj in second_seg_json:
            json.dump(json_obj, file)
            file.write('\n')

    print(f"Segments 1 & 3 saved to '{road_file_name}'")
    print(f"Segment 2 with wait times saved to '{segment_file_name}'")

Segments 1 & 3 saved to 'Segmented Trial Week/Segmented_Trips_01_25.json'
Segment 2 with wait times saved to 'Segmented Trial Week/Merchants_Segments_01_25.json'
Segments 1 & 3 saved to 'Segmented Trial Week/Segmented_Trips_01_26.json'
Segment 2 with wait times saved to 'Segmented Trial Week/Merchants_Segments_01_26.json'
Segments 1 & 3 saved to 'Segmented Trial Week/Segmented_Trips_01_27.json'
Segment 2 with wait times saved to 'Segmented Trial Week/Merchants_Segments_01_27.json'
Segments 1 & 3 saved to 'Segmented Trial Week/Segmented_Trips_01_28.json'
Segment 2 with wait times saved to 'Segmented Trial Week/Merchants_Segments_01_28.json'
Segments 1 & 3 saved to 'Segmented Trial Week/Segmented_Trips_01_29.json'
Segment 2 with wait times saved to 'Segmented Trial Week/Merchants_Segments_01_29.json'
Segments 1 & 3 saved to 'Segmented Trial Week/Segmented_Trips_01_30.json'
Segment 2 with wait times saved to 'Segmented Trial Week/Merchants_Segments_01_30.json'
Segments 1 & 3 saved to 'Seg

In [36]:
road_df.describe()

,dist,trip_time,weekID,timeID,dateID,time_offset,segmentID
count,64611.000000,64611.000000,64611.000000,64611.000000,64611.000000,64611.000000,64611.000000
mean,4.245647,545.493941,2.995945,882.030065,26.887450,462.391729,2.081147
std,25.907593,437.792605,1.946319,280.359040,2.044701,537.200286,0.996710
min,0.000000,0.000000,0.000000,3.000000,24.000000,0.000000,1.000000
25%,0.883395,208.500000,1.000000,686.000000,25.000000,0.000000,1.000000
50%,2.373664,432.000000,3.000000,904.200000,27.000000,344.000000,3.000000
75%,5.662286,780.000000,5.000000,1094.900000,29.000000,815.000000,3.000000
max,2872.321048,4980.000000,6.000000,1437.400000,30.000000,6442.000000,3.000000


In [42]:
second_segment_df.describe()

,dist,trip_time,weekID,timeID,dateID,time_offset
count,35009.0,35009.000000,35009.000000,35009.000000,35009.000000,35009.000000
mean,0.0,587.805764,2.981176,874.914708,26.907024,254.751550
std,0.0,392.575488,1.944636,282.744044,2.050928,251.245861
min,0.0,10.000000,0.000000,3.000000,24.000000,0.000000
25%,0.0,295.000000,1.000000,677.000000,25.000000,77.000000
50%,0.0,498.000000,3.000000,898.000000,27.000000,195.000000
75%,0.0,785.000000,5.000000,1090.000000,29.000000,358.000000
max,0.0,5824.000000,6.000000,1429.000000,30.000000,3415.000000
